<a href="https://colab.research.google.com/github/kplr-training/Web-Scraping/blob/main/Solution/4_Scrape_Car_Attributs_Using_Requests.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/kplr-training/Web-Scraping/blob/main/Solution/4_Scrape_Car_Attributs_Using_Requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Récupérer les attributs d'une voiture à l'aide de Requests**





- Nous allons extraire des données des sites web d'un concessionnaire automobile : nous allons extraire - la marque - le modèle - le type de carburant - les kilomètres - l'année du modèle - le prix.

![image](https://user-images.githubusercontent.com/123748165/224308731-cb43fb4b-69f9-4d5d-b52f-1d2b157aaae5.png)

#### 1 - Imports

In [ ]:
import requests 
import pandas as pd 
import sqlalchemy

#### 2 - CURL 

**Comment afficher les en-têtes HTTP ? [Y compris les cookies]**

- Avant de pouvoir utiliser les en-têtes dans notre code, nous devons être en mesure de les récupérer quelque part. Pour ce faire, utilisons notre propre navigateur et allons sur le site web cible. 
- Pour notre exemple, allons sur https://www.cazoo.co.uk/cars/?page=2 > clic droit > inspecter pour ouvrir les outils de développement.

- nous allons naviguer dans l'onglet Réseau et rechercher la requête "?page 2".

![image](https://user-images.githubusercontent.com/123748165/224308744-61c1ed97-1680-47bf-a531-0f0daa24f5ce.png)

- Clicking on the file we’ll open by default the Headers tab and by scrolling down we’ll see the Request Headers section.

![image](https://user-images.githubusercontent.com/123748165/224308758-b28b3301-4f75-42ac-a682-953790c4405c.png)


***Headers Arguments***

- **User-Agent** : Il s'agit probablement de l'en-tête le plus important, car il identifie "le type d'application, le système d'exploitation, le fournisseur du logiciel ou la version du logiciel de l'agent utilisateur demandeur".
le fournisseur du logiciel ou la version logicielle de l'agent utilisateur du logiciel demandeur", ce qui en fait le premier contrôle effectué par la plupart des serveurs.

- **Accept-Language** : Bien qu'elle ne soit pas toujours nécessaire, cette fonction indique au serveur la version linguistique des données à fournir. Lorsqu'il y a une grande différence entre chaque demande du point de vue de la langue, cela peut indiquer au serveur qu'un robot est impliqué.

- **Referer** : L'en-tête HTTP Referer indique au serveur la page d'où provient l'utilisateur. Bien qu'il soit principalement utilisé pour le suivi, il peut également nous aider à imiter le comportement d'un utilisateur organique, par exemple en indiquant au serveur que nous venons d'un moteur de recherche comme Google.

- **Cookie** : Les cookies permettent aux serveurs de communiquer à l'aide d'un petit morceau de données, mais que se passe-t-il lorsque le serveur envoie un cookie mais que le navigateur ne le stocke pas et ne le renvoie pas lors de la requête suivante ? Les cookies peuvent également être utilisés pour identifier si la demande provient d'un utilisateur réel ou d'un robot.
- **sec-fetch-site** : cet en-tête indique au serveur si une demande de ressource provient de la même origine, du même site, d'un site différent ou s'il s'agit d'une demande "initiée par l'utilisateur". Le serveur peut alors utiliser cette information pour décider si la demande doit être autorisée.
- **sec-fetch-mode** : permet à un serveur de faire la distinction entre : les demandes provenant d'un utilisateur naviguant entre des pages HTML, et les demandes de chargement d'images et d'autres ressources. Par exemple, cet en-tête contiendrait navigate pour les demandes de navigation de haut niveau, tandis que no-cors serait utilisé pour le chargement d'une image.

- **sec-fetch-Dest** : permet aux serveurs de déterminer s'ils doivent répondre à une demande en fonction de son utilisation prévue.

In [ ]:
headers = {
    'authority': 'www.cazoo.co.uk',
    'sec-ch-ua': '^\\^',
    'cazoo-language': 'en',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
    'cazoo-market': 'gb',
    'sec-ch-ua-platform': '^\\^Windows^\\^',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.cazoo.co.uk/cars/?page=2',
    'accept-language': 'en-US,en;q=0.9,de-DE;q=0.8,de;q=0.7',
    'cookie': 'base_device_id=d0a0a3f4-6853-469a-920f-6050e04a23d0; base_session_count=1; base_session_id=aa7da8a1-20b0-44f4-b994-7b19b379af45',
}

params = (
    ('sort', 'price-asc'),
)

response = requests.get('https://www.cazoo.co.uk/api/search', headers=headers, params=params)


3 - Check Status Code

In [ ]:
response

<Response [200]>

#### 4 - Créer un objet Json

In [ ]:
results_json = response.json()
results_json

#### 5 - Output Keys

In [ ]:
results_json.keys()

dict_keys(['results', 'filters', 'pagination'])

In [ ]:
listing = results_json['results']

#### 6 - Trouvez vos données 
- marque
- modèle
- type de carburant
- Kilométrage
- année du modèle
- prix

In [ ]:
# brand
listing[0]['make']

'Peugeot'

In [ ]:
# model
listing[0]['model']

'108'

In [ ]:
# fuel type
listing[0]['fuelType']['description']

'Petrol'

In [ ]:
#miles
listing[0]['mileage']

34207

In [ ]:
# model year
listing[0]['modelYear']

2015

In [ ]:
# price
listing[0]['pricing']['fullPrice']['value']

6550

#### 7 - Mettre tout ensemble - Boucler les résultats et ajouter des données dans une liste

In [ ]:
# empty lists
brand = []
model = []
fuel_type = []
miles = []
model_year = []
price = []

for result in listing:
    
    # brand
    brand.append(result['make'])
    
    # model 
    model.append(result['model'])
    
    # fuel type
    fuel_type.append(result['fuelType']['description'])
    
    # miles
    miles.append(result['mileage'])
    
    #model year
    model_year.append(result['modelYear'])
    
    # price
    price.append(result['pricing']['fullPrice']['value'])

#### 8 - Pandas Dataframe - Single Page

In [ ]:
car_df = pd.DataFrame({'Brand': brand, 'Model':model, 'Fuel Type': fuel_type, 'Mileage':miles,
                      'Model Year': model_year, 'Price':price})

In [ ]:
car_df

Brand            Model Fuel Type  Mileage  Model Year  Price
0      Peugeot              108    Petrol    34207        2015   6550
1        Dacia  Sandero Stepway    Petrol    65547        2012   6550
2       Suzuki          Celerio    Petrol    23190        2015   6650
3         Fiat              500    Petrol    41216        2015   6850
4        Dacia          Sandero    Petrol    10837        2012   6900
5         Ford               Ka    Petrol    34980        2015   6950
6       Suzuki          Celerio    Petrol    23778        2015   7050
7      Citroen               C1    Petrol    29950        2016   7150
8        Dacia          Sandero    Petrol    30780        2017   7150
9       Toyota             AYGO    Petrol    55659        2017   7250
10     Citroen       C3 Picasso    Petrol    61634        2016   7250
11         Kia          Picanto    Petrol    36057        2015   7300
12      Toyota             AYGO    Petrol    36463        2016   7350
13        Fiat              500    Petrol    35522        2015   7350
14       Dacia          Sandero    Petrol    27495        2017   7450
15    Vauxhall        Astra GTC    Petrol    42517        2015   7450
16    Vauxhall            Corsa    Petrol    38675        2016   7550
17    Vauxhall             Viva    Petrol    16550        2016   7550
18        Fiat             Tipo    Diesel    50445        2016   7550
19        Ford               Ka    Petrol    28293        2015   7600
20        Fiat              500    Petrol    38115        2015   7600
21        Ford           Fiesta    Petrol    61086        2017   7650
22     Peugeot              108    Petrol    14824        2015   7650
23        Fiat              500    Petrol    17693        2015   7700
24        Fiat              500    Petrol    33133        2017   7700
25    Vauxhall            Corsa    Petrol    62525        2017   7750
26        Fiat              500    Petrol    47463        2015   7750
27     Peugeot             2008    Petrol    52965        2015   7750
28  Volkswagen              up!    Petrol    13127        2015   7800
29      Toyota             AYGO    Petrol    33907        2015   7850
30      Toyota             AYGO    Petrol    36871        2018   7850
31      Toyota             AYGO    Petrol    31724        2016   7850
32      Toyota             AYGO    Petrol    22523        2015   7850
33          DS             DS 3    Petrol    40837        2015   7900
34        Ford           Fiesta    Petrol    64720        2016   7900
35        Fiat              500    Petrol    27119        2015   7950
36        Fiat              500    Petrol    21691        2015   7950
37        Fiat              500    Petrol    23006        2015   7950
38        Fiat              500    Petrol    28516        2015   7950
39    Vauxhall            Astra    Diesel    37435        2016   7950
40          DS             DS 3    Petrol    43589        2017   8000
41         Kia          Picanto    Petrol    29356        2015   8000
42        Seat            Ibiza    Petrol    91225        2018   8000
43       Smart          forfour    Petrol     5316        2018   8000
44        Ford           Fiesta    Petrol    59500        2016   8050
45        Ford           Fiesta    Petrol    41048        2016   8050
46       Smart          forfour    Petrol     7800        2018   8050
47    Vauxhall            Corsa    Petrol    59965        2017   8100

In [ ]:
car_df.to_csv('data.csv')

#### 9 - Scraping Plusieurs Pages (ici : 20 pages)

In [ ]:
headers = {
    'authority': 'www.cazoo.co.uk',
    'sec-ch-ua': '^\\^',
    'cazoo-language': 'en',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
    'cazoo-market': 'gb',
    'sec-ch-ua-platform': '^\\^Windows^\\^',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.cazoo.co.uk/cars/',
    'accept-language': 'en-US,en;q=0.9,de-DE;q=0.8,de;q=0.7',
    'cookie': 'base_device_id=d0a0a3f4-6853-469a-920f-6050e04a23d0; base_session_count=1',
}

# empty lists
brand = []
model = []
fuel_type = []
miles = []
model_year = []
price = []

for i in range(1,21):

    params = (
        ('page', str(i)),
        ('sort', 'price-asc'),
    )

    response = requests.get('https://www.cazoo.co.uk/api/search', headers=headers, params=params)

    results_json = response.json()
    
    listing = results_json['results']
    
    for result in listing:
    
        # brand
        try:
            brand.append(result['make'])
        except:
            brand.append('')

        # model 
        try:
            model.append(result['model'])
        except:
            model.append('')

        # fuel type
        try:
            fuel_type.append(result['fuelType']['description'])
        except:
            fuel_type.append('')

        # miles
        try:
            miles.append(result['mileage'])
        except:
            miles.append('')

        #model year
        try:
            model_year.append(result['modelYear'])
        except:
            model_year

        # price
        try:
            price.append(result['pricing']['fullPrice']['value'])
        except:
            price.append('')
        

car_df_multiple = pd.DataFrame({'Brand': brand, 'Model':model, 'Fuel Type': fuel_type, 'Mileage':miles,
                      'Model Year': model_year, 'Price':price})   

car_df_multiple


Brand            Model Fuel Type  Mileage  Model Year  Price
0    Peugeot              108    Petrol    34207        2015   6550
1      Dacia  Sandero Stepway    Petrol    65547        2012   6550
2     Suzuki          Celerio    Petrol    23190        2015   6650
3       Fiat              500    Petrol    41216        2015   6850
4      Dacia          Sandero    Petrol    10837        2012   6900
..       ...              ...       ...      ...         ...    ...
955     Fiat              500    Petrol    28935        2015  11600
956     Audi               A1    Diesel    48361        2016  11650
957   Nissan           Pulsar    Petrol    16518        2015  11650
958     Fiat              500    Petrol     5874        2018  11650
959  Hyundai              i20    Petrol     7917        2017  11650

[960 rows x 6 columns]

In [ ]:
car_df_multiple.to_csv('data_multiple.csv')